In [19]:
using Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/GitHub/dynamics-simulation-16-715/lecture-notebooks/Lecture 22/Project.toml`


In [20]:
using LinearAlgebra
using ForwardDiff
using Plots

In [21]:
#Parameters
m1 = 1.0
m2 = 1.0
J1 = Diagonal([0.1; 1.0; 1.0])
J2 = Diagonal([0.1; 1.0; 1.0])
ℓ1 = 1.0
ℓ2 = 1.0

1.0

In [22]:
M̄ = [m1*I(3) zeros(3,9);
     zeros(3,3) J1 zeros(3,6);
     zeros(3,6) m2*I(3) zeros(3,3);
     zeros(3,9) J2]

12×12 SparseArrays.SparseMatrixCSC{Float64, Int64} with 12 stored entries:
 1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   0.1   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   0.1   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0

In [23]:
#Simulation Parameters
h = 0.05 #20 Hz
Tf = 5.0 #final time (sec)
Thist = Array(0:h:Tf)
N = length(Thist)

101

In [24]:
function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I + hat(Q[2:4])]
end

function R(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I - hat(Q[2:4])]
end

H = [zeros(1,3); I];

T = Diagonal([1.0; -1; -1; -1])

function G(Q)
    return L(Q)*H
end

function Ḡ(q)
    Q1 = q[4:7]
    Q2 = q[11:14]
    
    return [I zeros(3,9); zeros(4,3) G(Q1) zeros(4,6); zeros(3,6) I zeros(3,3); zeros(4,9) G(Q2)]
end

Ḡ (generic function with 1 method)

In [25]:
function DEL(q_1,q_2,q_3,λ,F1,F2)
    
    r1_1 = q_1[1:3]
    Q1_1 = q_1[4:7]
    r2_1 = q_1[8:10]
    Q2_1 = q_1[11:14]
    
    r1_2 = q_2[1:3]
    Q1_2 = q_2[4:7]
    r2_2 = q_2[8:10]
    Q2_2 = q_2[11:14]
    
    r1_3 = q_3[1:3]
    Q1_3 = q_3[4:7]
    r2_3 = q_3[8:10]
    Q2_3 = q_3[11:14]
    
    [(1/h)*m1*(r1_2-r1_1) - (1/h)*m1*(r1_3-r1_2);
    (2.0/h)*G(Q1_2)'*L(Q1_1)*H*J1*H'*L(Q1_1)'*Q1_2 + (2.0/h)*G(Q1_2)'*T*R(Q1_3)'*H*J1*H'*L(Q1_2)'*Q1_3;
    (1/h)*m2*(r2_2-r2_1) - (1/h)*m2*(r2_3-r2_2);
    (2.0/h)*G(Q2_2)'*L(Q2_1)*H*J2*H'*L(Q2_1)'*Q2_2 + (2.0/h)*G(Q2_2)'*T*R(Q2_3)'*H*J2*H'*L(Q2_2)'*Q2_3] + (h/2.0)*F1 + (h/2.0)*F2 + h*Dc(q_2)'*λ
    
end

DEL (generic function with 1 method)

In [26]:
function Dq3DEL(q_1,q_2,q_3,λ,F1,F2)
    ForwardDiff.jacobian(dq->DEL(q_1,q_2,dq,λ,F1,F2), q_3)*Ḡ(q_3)
end

Dq3DEL (generic function with 1 method)

In [27]:
function c(q)
    r1 = q[1:3]
    Q1 = q[4:7]
    r2 = q[8:10]
    Q2 = q[11:14]
    
    [r1 + H'*R(Q1)'*L(Q1)*H*[0.5*ℓ1; 0; 0] - r2 - H'*R(Q2)'*L(Q2)*H*[-0.5*ℓ2; 0; 0];
        [0 1 0 0; 0 0 1 0]*L(Q1)'*Q2]
end

c (generic function with 1 method)

In [28]:
function Dc(q)
    ForwardDiff.jacobian(dq->c(dq),q)*Ḡ(q)
end

Dc (generic function with 1 method)

In [29]:
#initial conditions
r1_0 = zeros(3)
r2_0 = [1.0; 0; 0]
Q1_0 = [1.0; 0; 0; 0]
Q2_0 = [1.0; 0; 0; 0]

q_0 = [r1_0; Q1_0; r2_0; Q2_0]

14-element Vector{Float64}:
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0

In [31]:
#Torque input at joint
uhist = .5*[0; 0; ones(19); -ones(40); zeros(10); ones(40); -ones(10)];

#Corresponding F
Fhist = zeros(12,N)
for k = 1:N
    Fhist[:,k] = [0; 0; 0; 0; 0; -uhist[k]; 0; 0; 0; 0; 0; uhist[k]]
end

In [32]:
#Simulate
qhist = zeros(14,N)
qhist[:,1] .= q_0
qhist[:,2] .= q_0


for k = 2:(N-1)
    
    #Initial guess
    qhist[:,k+1] .= qhist[:,k]
    λ = zeros(5)
    
    e = [DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k]); c(qhist[:,k+1])]
    
    while maximum(abs.(e)) > 1e-12
        D = Dq3DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k])
        C2 = Dc(qhist[:,k])
        C3 = Dc(qhist[:,k+1])
        
        Δ = -[D h*C2'; C3 zeros(5,5)]\e
        
        qhist[1:3,k+1] .= qhist[1:3,k+1] + Δ[1:3]
        qhist[4:7,k+1] .= L(qhist[4:7,k+1])*[sqrt(1-Δ[4:6]'*Δ[4:6]); Δ[4:6]]
        qhist[8:10,k+1] .= qhist[8:10,k+1] + Δ[7:9]
        qhist[11:14,k+1] .= L(qhist[11:14,k+1])*[sqrt(1-Δ[10:12]'*Δ[10:12]); Δ[10:12]]
        
        λ .= λ + Δ[13:17]
        
        e = [DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k]); c(qhist[:,k+1])]
    end
    
end

In [33]:
using MeshCat, GeometryBasics, CoordinateTransformations, Rotations

In [34]:
vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /Users/zac/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8703))

In [35]:
delete!(vis)

green_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.8))
cylinder1 = Cylinder(Point(-.5, 0, 0), Point(0.5, 0, 0), 0.1)
setobject!(vis["cylinder1"],cylinder1,green_material)

red_material = MeshPhongMaterial(color=RGBA(1, 0, 0, 0.8))
cylinder2 = Cylinder(Point(-.5, 0, 0), Point(0.5, 0, 0), 0.1)
setobject!(vis["cylinder2"],cylinder2,red_material)

for k = 1:N
    
    # set position and attitude
    position1 = Translation(qhist[1:3,k]...)
    attitude1 = LinearMap(UnitQuaternion(qhist[4:7,k]))
    position2 = Translation(qhist[8:10,k]...)
    attitude2 = LinearMap(UnitQuaternion(qhist[11:14,k]))
    
    settransform!(vis["cylinder1"], compose(position1,attitude1))
    settransform!(vis["cylinder2"], compose(position2,attitude2))
    sleep(0.05)
end

In [36]:
0.5*(qhist[1:3,1] + qhist[8:10,1])

3-element Vector{Float64}:
 0.5
 0.0
 0.0

In [37]:
0.5*(qhist[1:3,end] + qhist[8:10,end])

3-element Vector{Float64}:
 0.5000000000000083
 0.0
 0.0